In [1]:
import numpy as np
import scipy as sp
import csi
from hcsi import *

In [91]:
inp = csi.loadData('testdata/dream4-csidata_normalised.csv')

In [3]:
args = parse_args()

usage: __main__.py [-h] --Input INPUT [--Depth DEPTH] [--Prior GPPRIOR]
                   [--BetaPrior BETAPRIOR] [--Genes GENES [GENES ...]]
                   [--Pool POOL] [--Samples SAMPLES] [--BurnIn BURNIN]
                   [--Pickle]
__main__.py: error: the following arguments are required: --Input


SystemExit: 2

To exit: use 'exit', 'quit', or Ctrl-D.


In [10]:
def parse_args():
	#the obligatory argument parsing
	parser = argparse.ArgumentParser()
	parser.add_argument('--Input', dest='input', type=argparse.FileType('r'), required=True, help='The CSV file featuring your expression data. First column for gene names, first row for condition names repeated for each condition time point, second row for condition time point information.')
	parser.add_argument('--Depth', dest='depth', type=int, default=2, help='CSI parental set depth truncation to maintain computational tractability. Default: 2')
	parser.add_argument('--Prior', dest='gpprior', type=str, default='10,0.1', help='CSI Gaussian Process prior, provided as \'shape,scale\' for a gamma distribution or \'uniform\' for a uniform distribution. Default: \'10,0.1\'')
	parser.add_argument('--BetaPrior', dest='betaprior', type=str, default='1,1', help='hCSI temperature prior, provided as \'shape,scale\' for a gamma distribution. Default: \'1,1\'')
	parser.add_argument('--Genes', dest='genes', default=None, nargs='+', help='Child gene set to evaluate, if you wish to only run hCSI on a subset of the available gene space. Provide as space delimited names matching the CSV file. Default: None (analyse the whole dataset)')
	parser.add_argument('--Pool', dest='pool', type=int, default=1, help='Number of threads to open up for parallelising hCSI on a per-gene basis. Default: 1 (no parallelising)')
	parser.add_argument('--Samples', dest='samples', type=int, default=100000, help='Number of Gibbs updates to perform within hCSI. Default: 100,000')
	parser.add_argument('--BurnIn', dest='burnin', type=int, default=10000, help='Number of initial Gibbs updates to discard as burn-in. Default: 10,000')
	parser.add_argument('--Pickle', dest='pickle', action='store_true', help='Flag. If provided, the obtained Gibbs value chains for individual models and the hypernetwork are stored as a Python Pickle. Refer to readme for more in depth formatting information.')
	args = parser.parse_args(['--Depth','2','--Input','testdata/Demo_DREAM.csv'])
	return args

In [11]:
args = parse_args()

In [12]:
args

Namespace(betaprior='1,1', burnin=10000, depth=2, genes=None, gpprior='10,0.1', input=<_io.TextIOWrapper name='testdata/Demo_DREAM.csv' mode='r' encoding='UTF-8'>, pickle=False, pool=1, samples=100000)

In [13]:
betas = np.ones((5,1))

In [14]:
cc = csi.Csi(inp)

In [15]:
vals = cc.allParents('G1',2)

In [16]:
vals

[([], 'G1'),
 (['G2'], 'G1'),
 (['G3'], 'G1'),
 (['G4'], 'G1'),
 (['G5'], 'G1'),
 (['G6'], 'G1'),
 (['G7'], 'G1'),
 (['G8'], 'G1'),
 (['G9'], 'G1'),
 (['G10'], 'G1'),
 (['G2', 'G3'], 'G1'),
 (['G2', 'G4'], 'G1'),
 (['G2', 'G5'], 'G1'),
 (['G2', 'G6'], 'G1'),
 (['G2', 'G7'], 'G1'),
 (['G2', 'G8'], 'G1'),
 (['G2', 'G9'], 'G1'),
 (['G2', 'G10'], 'G1'),
 (['G3', 'G4'], 'G1'),
 (['G3', 'G5'], 'G1'),
 (['G3', 'G6'], 'G1'),
 (['G3', 'G7'], 'G1'),
 (['G3', 'G8'], 'G1'),
 (['G3', 'G9'], 'G1'),
 (['G3', 'G10'], 'G1'),
 (['G4', 'G5'], 'G1'),
 (['G4', 'G6'], 'G1'),
 (['G4', 'G7'], 'G1'),
 (['G4', 'G8'], 'G1'),
 (['G4', 'G9'], 'G1'),
 (['G4', 'G10'], 'G1'),
 (['G5', 'G6'], 'G1'),
 (['G5', 'G7'], 'G1'),
 (['G5', 'G8'], 'G1'),
 (['G5', 'G9'], 'G1'),
 (['G5', 'G10'], 'G1'),
 (['G6', 'G7'], 'G1'),
 (['G6', 'G8'], 'G1'),
 (['G6', 'G9'], 'G1'),
 (['G6', 'G10'], 'G1'),
 (['G7', 'G8'], 'G1'),
 (['G7', 'G9'], 'G1'),
 (['G7', 'G10'], 'G1'),
 (['G8', 'G9'], 'G1'),
 (['G8', 'G10'], 'G1'),
 (['G9', 'G10'], 'G1'

In [17]:
condParents = [vals[1],vals[1],vals[1],vals[1],vals[1]]

In [19]:
        i = 0
        distribution = np.zeros((len(vals),1))
        for v in vals:
            p = 1
            for (j, cond) in enumerate(condParents):
                p = p * np.exp((-1)*betas[j]*hamming(v,cond))
            distribution[i] = p
            i = i + 1

In [20]:
distribution

array([[  6.73794700e-03],
       [  1.00000000e+00],
       [  4.53999298e-05],
       [  4.53999298e-05],
       [  4.53999298e-05],
       [  4.53999298e-05],
       [  4.53999298e-05],
       [  4.53999298e-05],
       [  4.53999298e-05],
       [  4.53999298e-05],
       [  6.73794700e-03],
       [  6.73794700e-03],
       [  6.73794700e-03],
       [  6.73794700e-03],
       [  6.73794700e-03],
       [  6.73794700e-03],
       [  6.73794700e-03],
       [  6.73794700e-03],
       [  3.05902321e-07],
       [  3.05902321e-07],
       [  3.05902321e-07],
       [  3.05902321e-07],
       [  3.05902321e-07],
       [  3.05902321e-07],
       [  3.05902321e-07],
       [  3.05902321e-07],
       [  3.05902321e-07],
       [  3.05902321e-07],
       [  3.05902321e-07],
       [  3.05902321e-07],
       [  3.05902321e-07],
       [  3.05902321e-07],
       [  3.05902321e-07],
       [  3.05902321e-07],
       [  3.05902321e-07],
       [  3.05902321e-07],
       [  3.05902321e-07],
 

In [54]:
np.random.seed(1)

In [26]:
gpprior = [10,0.1]
betaprior = [1,1]

In [53]:
gene = 'G1'

In [55]:
	hnrv = RandomVariableHyperNetwork(inp,gene,args.depth)
	crv = []
	conditions = np.unique([x[0] for x in inp.columns.values])
	for cond in conditions:
		crv.append(RandomVariableCondition(inp,cond,gene,gpprior,betaprior,args.depth))
	gibbs = GibbsHCSI(crv,hnrv)
	gibbs.sample(100)

In [25]:
gpprior[0]

10.0

In [56]:
gibbs.sampledValuesHyperNetwork

[([], 'G1'),
 ([], 'G1'),
 ([], 'G1'),
 ([], 'G1'),
 (['G9'], 'G1'),
 (['G3'], 'G1'),
 (['G4', 'G9'], 'G1'),
 (['G4', 'G7'], 'G1'),
 (['G4', 'G7'], 'G1'),
 (['G6', 'G8'], 'G1'),
 (['G9'], 'G1'),
 (['G6'], 'G1'),
 (['G2', 'G3'], 'G1'),
 (['G3', 'G9'], 'G1'),
 (['G8', 'G10'], 'G1'),
 (['G3', 'G6'], 'G1'),
 (['G3', 'G7'], 'G1'),
 (['G9'], 'G1'),
 (['G5'], 'G1'),
 (['G3', 'G7'], 'G1'),
 (['G4', 'G5'], 'G1'),
 (['G7', 'G10'], 'G1'),
 (['G9'], 'G1'),
 (['G9', 'G10'], 'G1'),
 (['G6', 'G8'], 'G1'),
 (['G7'], 'G1'),
 (['G5', 'G10'], 'G1'),
 (['G2', 'G8'], 'G1'),
 (['G5', 'G9'], 'G1'),
 (['G9'], 'G1'),
 ([], 'G1'),
 (['G7', 'G9'], 'G1'),
 (['G7', 'G8'], 'G1'),
 (['G10'], 'G1'),
 (['G2', 'G7'], 'G1'),
 (['G3', 'G4'], 'G1'),
 (['G7'], 'G1'),
 (['G10'], 'G1'),
 (['G9', 'G10'], 'G1'),
 (['G8', 'G9'], 'G1'),
 (['G4', 'G9'], 'G1'),
 (['G2', 'G8'], 'G1'),
 (['G2', 'G10'], 'G1'),
 (['G4', 'G6'], 'G1'),
 (['G9'], 'G1'),
 (['G2'], 'G1'),
 (['G4', 'G7'], 'G1'),
 (['G5', 'G7'], 'G1'),
 (['G3', 'G7'], 'G1'),

In [52]:
gibbs2.sampledValuesHyperNetwork

[(['G5'], 'G2'),
 (['G5'], 'G2'),
 (['G9'], 'G2'),
 (['G5'], 'G2'),
 (['G5'], 'G2'),
 (['G3'], 'G2'),
 (['G8', 'G9'], 'G2'),
 (['G5', 'G6'], 'G2'),
 (['G5', 'G7'], 'G2'),
 (['G6', 'G8'], 'G2'),
 (['G9'], 'G2'),
 (['G7'], 'G2'),
 (['G1', 'G4'], 'G2'),
 (['G3', 'G10'], 'G2'),
 (['G8', 'G10'], 'G2'),
 (['G3', 'G5'], 'G2'),
 (['G3', 'G7'], 'G2'),
 (['G10'], 'G2'),
 (['G5'], 'G2'),
 (['G3', 'G8'], 'G2'),
 (['G4', 'G7'], 'G2'),
 (['G7', 'G10'], 'G2'),
 (['G1', 'G3'], 'G2'),
 (['G9', 'G10'], 'G2'),
 (['G6', 'G8'], 'G2'),
 (['G7'], 'G2'),
 (['G6', 'G8'], 'G2'),
 (['G1', 'G9'], 'G2'),
 (['G5', 'G10'], 'G2'),
 (['G10'], 'G2'),
 ([], 'G2'),
 (['G8', 'G9'], 'G2'),
 (['G7', 'G8'], 'G2'),
 (['G10'], 'G2'),
 (['G1', 'G7'], 'G2'),
 (['G3', 'G6'], 'G2'),
 (['G7'], 'G2'),
 (['G10'], 'G2'),
 (['G9', 'G10'], 'G2'),
 (['G8', 'G10'], 'G2'),
 (['G4', 'G7'], 'G2'),
 (['G1', 'G8'], 'G2'),
 (['G3', 'G5'], 'G2'),
 (['G4', 'G7'], 'G2'),
 (['G9'], 'G2'),
 (['G1'], 'G2'),
 (['G4', 'G6'], 'G2'),
 (['G5', 'G7'], 'G2'

In [58]:
hamming(gibbs.sampledValuesHyperNetwork[-4],gibbs.sampledValuesHyperNetwork[-1])

2

In [60]:
        valList = []
        betaList = []
        pset = gibbs.rvHyperNetwork.getValRange()
        for i in gibbs.indexList:
            #for each variable, build a distribution accross all possible values for that variable
            #based on current values of other variable 
            distribution = gibbs.rvl[i].getConditionalDistribution(gibbs.rvHyperNetwork.getCurrentValue())
            #sample a random value based on the distribution
            valInd = np.random.choice(np.arange(len(pset)),size=1,p=distribution)
            gibbs.rvl[i].setCurrentValue(pset[valInd[0]])
            valList.append(pset[valInd[0]])
            betaList.append(gibbs.rvl[i].beta)

In [61]:
valList

[(['G3', 'G4'], 'G1'),
 (['G2'], 'G1'),
 (['G2', 'G9'], 'G1'),
 (['G7', 'G9'], 'G1'),
 (['G3'], 'G1')]

In [65]:
distribution

array([ 0.11033623,  0.0178858 ,  0.03489187,  0.04372361,  0.12343093,
        0.01481586,  0.0257556 ,  0.01467805,  0.07241652,  0.02970135,
        0.00415073,  0.00659708,  0.01989574,  0.00151504,  0.00382595,
        0.00160004,  0.00974229,  0.00523349,  0.01526926,  0.04173943,
        0.00331059,  0.0092868 ,  0.00430969,  0.02357789,  0.00786012,
        0.05279219,  0.00527499,  0.0110269 ,  0.00511318,  0.03650867,
        0.01308517,  0.01544649,  0.02601102,  0.01199648,  0.05997833,
        0.03216603,  0.00836828,  0.00673207,  0.01150554,  0.00379757,
        0.00582905,  0.0196216 ,  0.00688386,  0.00691068,  0.00266589,
        0.01273604])

In [68]:
np.std(sp.stats.mstats.zscore(np.arange(10)))

1.0

In [74]:
sp.stats.mstats.zscore(inp,axis=1,ddof=1)

array([[ 1.14614068, -0.83196402, -1.43941838, ...,  1.18681392,
         1.24286821,  0.90418191],
       [-0.29154641,  2.12234418,  1.71160192, ...,  1.16957892,
         0.85679254,  0.59747673],
       [ 1.76295457,  1.32858122,  0.8122826 , ...,  0.33250342,
         0.34876355,  0.85990218],
       ..., 
       [ 0.8132933 ,  1.45481444,  0.48754648, ...,  1.43382361,
         1.32435871,  2.0886676 ],
       [-0.92121831, -0.97141206, -1.0500507 , ..., -0.72588622,
        -0.83435533, -0.84326367],
       [ 0.45477904, -1.36723735, -1.76304685, ...,  1.08985451,
         0.7085861 ,  1.63948287]])

In [73]:
np.std(inp.iloc[0,:],ddof=1)

1.0000000000135358

In [75]:
inp

condition         A                                                    \
time           0         50        100       150       200       250    
G1         1.146141 -0.831964 -1.439418 -1.737505 -1.749940 -1.756526   
G2        -0.291546  2.122344  1.711602 -0.043294  0.554139  1.055807   
G3         1.762955  1.328581  0.812283  0.977008  0.497170  0.238698   
G4        -0.761209 -0.790224 -0.757698 -0.750799 -0.735785 -0.350352   
G5        -0.626803  0.283999  0.150912  0.460401  0.334145  0.272127   
G6        -0.859092 -0.973677 -0.500515 -0.023210  0.519307  0.808823   
G7        -0.654526 -0.656295 -0.687107 -0.678800 -0.539539 -0.544813   
G8         0.813293  1.454814  0.487546  0.283878 -0.178654 -0.622127   
G9        -0.921218 -0.971412 -1.050051 -1.362399 -0.997308 -0.742705   
G10        0.454779 -1.367237 -1.763047 -1.945656 -1.997908 -2.110899   

condition                                            ...            E  \
time           300       350       400       450     ...         550    
G1        -1.765812 -1.769060 -1.760694 -1.759075    ...     1.514828   
G2         0.679701  0.786423  0.448707 -0.105750    ...     0.261458   
G3         0.045474  0.304781 -0.127848  0.642854    ...     0.198277   
G4        -0.641694 -0.265993 -0.757629 -0.322865    ...    -0.442139   
G5         0.280810  0.390443  0.485875  0.445135    ...    -0.494777   
G6         1.028765  1.231111  1.315269  1.422283    ...    -0.302052   
G7        -0.173020 -0.097119 -0.090761  0.042880    ...    -0.199461   
G8        -0.883580 -0.554871 -0.800245 -0.777222    ...    -0.475700   
G9        -0.155355  0.624382  1.433256  1.058768    ...     0.116064   
G10       -1.964892 -1.882316 -1.978326 -2.063066    ...     0.242996   

condition                                                              \
time           600       650       700       750       800       850    
G1         0.534294 -0.024755 -0.126884  0.600194  0.914042  0.786232   
G2         0.493261  0.377926  0.753657  0.385389  0.574857  0.281021   
G3         1.033614  0.204734  1.029577  0.853479  0.498641  0.719518   
G4        -0.642049 -0.146133 -0.377510 -0.774507 -0.786612 -0.219383   
G5        -0.257971 -0.512780 -0.653853 -0.469474 -0.591243 -0.697342   
G6        -0.742568 -0.592456 -0.749805 -0.901809 -0.858331 -0.881814   
G7         0.085773 -0.518966 -0.526747 -0.592528 -0.560815 -0.575871   
G8         0.281614  0.589848  1.161354  0.836138  1.216442  0.984242   
G9         1.098921  0.282571 -0.432951 -0.723064 -0.921609 -0.913787   
G10        0.364398  0.499968  0.172150  0.541871  0.641857  0.955349   

condition                                
time           900       950       1000  
G1         1.186814  1.242868  0.904182  
G2         1.169579  0.856793  0.597477  
G3         0.332503  0.348764  0.859902  
G4        -0.745372 -0.752894 -0.737860  
G5        -0.451460 -0.530345 -0.429491  
G6        -0.895345 -0.897960 -0.932892  
G7        -0.693039 -0.693202 -0.681000  
G8         1.433824  1.324359  2.088668  
G9        -0.725886 -0.834355 -0.843264  
G10        1.089855  0.708586  1.639483  

[10 rows x 105 columns]

In [76]:
inp2 = inp

In [89]:
inp2.set_value(inp2.index,inp2.columns,inp2-7)

condition         A                                                    \
time           0         50        100       150       200       250    
G1         1.146141 -0.831964 -1.439418 -1.737505 -1.749940 -1.756526   
G2         1.146141 -0.831964 -1.439418 -1.737505 -1.749940 -1.756526   
G3        -0.291546  2.122344  1.711602 -0.043294  0.554139  1.055807   
G4        -0.291546  2.122344  1.711602 -0.043294  0.554139  1.055807   
G5         1.762955  1.328581  0.812283  0.977008  0.497170  0.238698   
G6         1.762955  1.328581  0.812283  0.977008  0.497170  0.238698   
G7        -0.761209 -0.790224 -0.757698 -0.750799 -0.735785 -0.350352   
G8        -0.761209 -0.790224 -0.757698 -0.750799 -0.735785 -0.350352   
G9        -0.626803  0.283999  0.150912  0.460401  0.334145  0.272127   
G10       -0.626803  0.283999  0.150912  0.460401  0.334145  0.272127   

condition                                            ...            E  \
time           300       350       400       450     ...         550    
G1        -1.765812 -1.769060 -1.760694 -1.759075    ...     1.514828   
G2        -1.765812 -1.769060 -1.760694 -1.759075    ...     1.514828   
G3         0.679701  0.786423  0.448707 -0.105750    ...     0.261458   
G4         0.679701  0.786423  0.448707 -0.105750    ...     0.261458   
G5         0.045474  0.304781 -0.127848  0.642854    ...     0.198277   
G6         0.045474  0.304781 -0.127848  0.642854    ...     0.198277   
G7        -0.641694 -0.265993 -0.757629 -0.322865    ...    -0.442139   
G8        -0.641694 -0.265993 -0.757629 -0.322865    ...    -0.442139   
G9         0.280810  0.390443  0.485875  0.445135    ...    -0.494777   
G10        0.280810  0.390443  0.485875  0.445135    ...    -0.494777   

condition                                                              \
time           600       650       700       750       800       850    
G1         0.534294 -0.024755 -0.126884  0.600194  0.914042  0.786232   
G2         0.534294 -0.024755 -0.126884  0.600194  0.914042  0.786232   
G3         0.493261  0.377926  0.753657  0.385389  0.574857  0.281021   
G4         0.493261  0.377926  0.753657  0.385389  0.574857  0.281021   
G5         1.033614  0.204734  1.029577  0.853479  0.498641  0.719518   
G6         1.033614  0.204734  1.029577  0.853479  0.498641  0.719518   
G7        -0.642049 -0.146133 -0.377510 -0.774507 -0.786612 -0.219383   
G8        -0.642049 -0.146133 -0.377510 -0.774507 -0.786612 -0.219383   
G9        -0.257971 -0.512780 -0.653853 -0.469474 -0.591243 -0.697342   
G10       -0.257971 -0.512780 -0.653853 -0.469474 -0.591243 -0.697342   

condition                                
time           900       950       1000  
G1         1.186814  1.242868  0.904182  
G2         1.186814  1.242868  0.904182  
G3         1.169579  0.856793  0.597477  
G4         1.169579  0.856793  0.597477  
G5         0.332503  0.348764  0.859902  
G6         0.332503  0.348764  0.859902  
G7        -0.745372 -0.752894 -0.737860  
G8        -0.745372 -0.752894 -0.737860  
G9        -0.451460 -0.530345 -0.429491  
G10       -0.451460 -0.530345 -0.429491  

[10 rows x 105 columns]

In [90]:
inp

condition         A                                                    \
time           0         50        100       150       200       250    
G1         1.146141 -0.831964 -1.439418 -1.737505 -1.749940 -1.756526   
G2         1.146141 -0.831964 -1.439418 -1.737505 -1.749940 -1.756526   
G3        -0.291546  2.122344  1.711602 -0.043294  0.554139  1.055807   
G4        -0.291546  2.122344  1.711602 -0.043294  0.554139  1.055807   
G5         1.762955  1.328581  0.812283  0.977008  0.497170  0.238698   
G6         1.762955  1.328581  0.812283  0.977008  0.497170  0.238698   
G7        -0.761209 -0.790224 -0.757698 -0.750799 -0.735785 -0.350352   
G8        -0.761209 -0.790224 -0.757698 -0.750799 -0.735785 -0.350352   
G9        -0.626803  0.283999  0.150912  0.460401  0.334145  0.272127   
G10       -0.626803  0.283999  0.150912  0.460401  0.334145  0.272127   

condition                                            ...            E  \
time           300       350       400       450     ...         550    
G1        -1.765812 -1.769060 -1.760694 -1.759075    ...     1.514828   
G2        -1.765812 -1.769060 -1.760694 -1.759075    ...     1.514828   
G3         0.679701  0.786423  0.448707 -0.105750    ...     0.261458   
G4         0.679701  0.786423  0.448707 -0.105750    ...     0.261458   
G5         0.045474  0.304781 -0.127848  0.642854    ...     0.198277   
G6         0.045474  0.304781 -0.127848  0.642854    ...     0.198277   
G7        -0.641694 -0.265993 -0.757629 -0.322865    ...    -0.442139   
G8        -0.641694 -0.265993 -0.757629 -0.322865    ...    -0.442139   
G9         0.280810  0.390443  0.485875  0.445135    ...    -0.494777   
G10        0.280810  0.390443  0.485875  0.445135    ...    -0.494777   

condition                                                              \
time           600       650       700       750       800       850    
G1         0.534294 -0.024755 -0.126884  0.600194  0.914042  0.786232   
G2         0.534294 -0.024755 -0.126884  0.600194  0.914042  0.786232   
G3         0.493261  0.377926  0.753657  0.385389  0.574857  0.281021   
G4         0.493261  0.377926  0.753657  0.385389  0.574857  0.281021   
G5         1.033614  0.204734  1.029577  0.853479  0.498641  0.719518   
G6         1.033614  0.204734  1.029577  0.853479  0.498641  0.719518   
G7        -0.642049 -0.146133 -0.377510 -0.774507 -0.786612 -0.219383   
G8        -0.642049 -0.146133 -0.377510 -0.774507 -0.786612 -0.219383   
G9        -0.257971 -0.512780 -0.653853 -0.469474 -0.591243 -0.697342   
G10       -0.257971 -0.512780 -0.653853 -0.469474 -0.591243 -0.697342   

condition                                
time           900       950       1000  
G1         1.186814  1.242868  0.904182  
G2         1.186814  1.242868  0.904182  
G3         1.169579  0.856793  0.597477  
G4         1.169579  0.856793  0.597477  
G5         0.332503  0.348764  0.859902  
G6         0.332503  0.348764  0.859902  
G7        -0.745372 -0.752894 -0.737860  
G8        -0.745372 -0.752894 -0.737860  
G9        -0.451460 -0.530345 -0.429491  
G10       -0.451460 -0.530345 -0.429491  

[10 rows x 105 columns]

In [85]:
inp2.columns

MultiIndex(levels=[['A', 'B', 'C', 'D', 'E'], [0.0, 50.0, 100.0, 150.0, 200.0, 250.0, 300.0, 350.0, 400.0, 450.0, 500.0, 550.0, 600.0, 650.0, 700.0, 750.0, 800.0, 850.0, 900.0, 950.0, 1000.0]],
           labels=[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]],
           names=['condition', 'time'])

In [92]:
csidata = inp

In [93]:
        colNames = np.asarray([x[0] for x in csidata.columns.values])
        numtemp = np.arange(len(colNames))
        inp = csidata.iloc[:,numtemp[colNames=='A']]

In [94]:
inp

condition         A                                                    \
time           0         50        100       150       200       250    
G1         1.146141 -0.831964 -1.439418 -1.737505 -1.749940 -1.756526   
G2        -0.291546  2.122344  1.711602 -0.043294  0.554139  1.055807   
G3         1.762955  1.328581  0.812283  0.977008  0.497170  0.238698   
G4        -0.761209 -0.790224 -0.757698 -0.750799 -0.735785 -0.350352   
G5        -0.626803  0.283999  0.150912  0.460401  0.334145  0.272127   
G6        -0.859092 -0.973677 -0.500515 -0.023210  0.519307  0.808823   
G7        -0.654526 -0.656295 -0.687107 -0.678800 -0.539539 -0.544813   
G8         0.813293  1.454814  0.487546  0.283878 -0.178654 -0.622127   
G9        -0.921218 -0.971412 -1.050051 -1.362399 -0.997308 -0.742705   
G10        0.454779 -1.367237 -1.763047 -1.945656 -1.997908 -2.110899   

condition                                            ...               \
time           300       350       400       450     ...         550    
G1        -1.765812 -1.769060 -1.760694 -1.759075    ...    -1.065562   
G2         0.679701  0.786423  0.448707 -0.105750    ...    -0.333464   
G3         0.045474  0.304781 -0.127848  0.642854    ...    -1.384221   
G4        -0.641694 -0.265993 -0.757629 -0.322865    ...    -0.235745   
G5         0.280810  0.390443  0.485875  0.445135    ...    -0.314326   
G6         1.028765  1.231111  1.315269  1.422283    ...     0.898790   
G7        -0.173020 -0.097119 -0.090761  0.042880    ...     0.185253   
G8        -0.883580 -0.554871 -0.800245 -0.777222    ...    -0.652406   
G9        -0.155355  0.624382  1.433256  1.058768    ...     0.703619   
G10       -1.964892 -1.882316 -1.978326 -2.063066    ...     0.479335   

condition                                                              \
time           600       650       700       750       800       850    
G1        -0.649659 -0.629173 -0.257208 -0.380672  0.463315  1.037239   
G2        -0.506347 -1.291891 -0.237260  0.641637  1.322777  0.635004   
G3        -0.903736 -0.125114 -0.058419  0.459535  0.719011  0.510177   
G4         1.214480  0.767000 -0.595829 -0.442436 -0.619828 -0.610809   
G5        -0.261582 -0.468760 -0.432408 -0.566442 -0.577498 -0.370895   
G6         0.604090 -0.177900 -0.699485 -0.755756 -0.863515 -0.822174   
G7        -0.198905 -0.305585 -0.463182 -0.557262 -0.532882 -0.687517   
G8        -0.283706 -0.447781  0.498545  0.733694  0.815103  0.710402   
G9         0.557734  0.516954  1.039810 -0.011707 -0.405123 -0.862391   
G10        0.336529  0.970085  1.058936  1.072676  0.611052  0.628614   

condition                                
time           900       950       1000  
G1         1.083882  1.288398  1.411513  
G2         0.631346  0.485557  2.338577  
G3         1.030492  1.134261  0.476418  
G4        -0.980726 -0.662609 -0.783554  
G5        -0.725658 -0.682111 -0.603460  
G6        -0.900635 -0.861065 -0.898248  
G7        -0.420166 -0.658018 -0.686545  
G8         0.920479  1.075660  1.583848  
G9        -0.730984 -0.996581 -0.618690  
G10        0.480049  0.452110  1.496374  

[10 rows x 21 columns]

In [95]:
inp.set_value(inp.index,inp.columns,sp.stats.mstats.zscore(inp,axis=1,ddof=1))

//anaconda/lib/python3.4/site-packages/pandas/core/indexing.py:420: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


condition         A                                                    \
time           0         50        100       150       200       250    
G1         1.457348 -0.181595 -0.684896 -0.931874 -0.942176 -0.947633   
G2        -0.947806  1.801261  1.333486 -0.665083  0.015306  0.586631   
G3         1.900531  1.297516  0.580768  0.809447  0.143316 -0.215506   
G4        -0.587374 -0.642793 -0.580669 -0.567492 -0.538813  0.197373   
G5        -1.141544  0.828327  0.540489  1.209848  0.936783  0.802652   
G6        -0.969230 -1.091449 -0.586762 -0.077659  0.501003  0.809808   
G7        -0.899677 -0.906224 -1.020254 -0.989509 -0.474129 -0.493646   
G8         0.804977  1.587897  0.407431  0.158872 -0.405608 -0.946827   
G9        -0.889359 -0.945905 -1.034494 -1.386365 -0.975077 -0.688258   
G10        0.708361 -0.615681 -0.903312 -1.036012 -1.073984 -1.156093   

condition                                            ...               \
time           300       350       400       450     ...         550    
G1        -0.955327 -0.958018 -0.951086 -0.949745    ...    -0.375141   
G2         0.158302  0.279842 -0.104767 -0.736211    ...    -0.995543   
G3        -0.483748 -0.123766 -0.724361  0.345561    ...    -2.468511   
G4        -0.359098  0.358499 -0.580537  0.249874    ...     0.416273   
G5         0.821430  1.058544  1.264942  1.176831    ...    -0.465723   
G6         1.044403  1.260230  1.349995  1.464139    ...     0.905768   
G7         0.882302  1.163199  1.186729  1.681313    ...     2.208214   
G8        -1.265907 -0.864747 -1.164204 -1.136106    ...    -0.983780   
G9        -0.026587  0.851815  1.763040  1.341167    ...     0.941079   
G10       -1.049991 -0.989983 -1.059754 -1.121333    ...     0.726206   

condition                                                              \
time           600       650       700       750       800       850    
G1        -0.030548 -0.013574  0.294615  0.192319  0.891598  1.367119   
G2        -1.192432 -2.087052 -0.885982  0.114952  0.890671  0.107399   
G3        -1.801480 -0.720565 -0.627976  0.091069  0.451285  0.161373   
G4         3.186237  2.331540 -0.271496  0.021489 -0.317333 -0.300106   
G5        -0.351648 -0.799730 -0.721109 -1.010996 -1.034908 -0.588070   
G6         0.591435 -0.242654 -0.798989 -0.859009 -0.973947 -0.929852   
G7         0.786504  0.391701 -0.191543 -0.539719 -0.449491 -1.021770   
G8        -0.533814 -0.734053  0.420854  0.707833  0.807185  0.679407   
G9         0.776734  0.730794  1.319809  0.135237 -0.307959 -0.823089   
G10        0.622430  1.082830  1.147397  1.157381  0.821923  0.834685   

condition                                
time           900       950       1000  
G1         1.405764  1.575214  1.677220  
G2         0.103234 -0.062799  2.047518  
G3         0.883697  1.027753  0.114507  
G4        -1.006657 -0.399047 -0.630055  
G5        -1.355346 -1.261162 -1.091058  
G6        -1.013540 -0.971334 -1.010994  
G7        -0.032347 -0.912598 -1.018173  
G8         0.935787  1.125172  1.745371  
G9        -0.675054 -0.974258 -0.548550  
G10        0.726725  0.706422  1.465279  

[10 rows x 21 columns]

In [98]:
np.std(inp.iloc[0,:],ddof=1)

1.0000000000000002

In [99]:
inp.iloc[:,:]

condition         A                                                    \
time           0         50        100       150       200       250    
G1         1.457348 -0.181595 -0.684896 -0.931874 -0.942176 -0.947633   
G2        -0.947806  1.801261  1.333486 -0.665083  0.015306  0.586631   
G3         1.900531  1.297516  0.580768  0.809447  0.143316 -0.215506   
G4        -0.587374 -0.642793 -0.580669 -0.567492 -0.538813  0.197373   
G5        -1.141544  0.828327  0.540489  1.209848  0.936783  0.802652   
G6        -0.969230 -1.091449 -0.586762 -0.077659  0.501003  0.809808   
G7        -0.899677 -0.906224 -1.020254 -0.989509 -0.474129 -0.493646   
G8         0.804977  1.587897  0.407431  0.158872 -0.405608 -0.946827   
G9        -0.889359 -0.945905 -1.034494 -1.386365 -0.975077 -0.688258   
G10        0.708361 -0.615681 -0.903312 -1.036012 -1.073984 -1.156093   

condition                                            ...               \
time           300       350       400       450     ...         550    
G1        -0.955327 -0.958018 -0.951086 -0.949745    ...    -0.375141   
G2         0.158302  0.279842 -0.104767 -0.736211    ...    -0.995543   
G3        -0.483748 -0.123766 -0.724361  0.345561    ...    -2.468511   
G4        -0.359098  0.358499 -0.580537  0.249874    ...     0.416273   
G5         0.821430  1.058544  1.264942  1.176831    ...    -0.465723   
G6         1.044403  1.260230  1.349995  1.464139    ...     0.905768   
G7         0.882302  1.163199  1.186729  1.681313    ...     2.208214   
G8        -1.265907 -0.864747 -1.164204 -1.136106    ...    -0.983780   
G9        -0.026587  0.851815  1.763040  1.341167    ...     0.941079   
G10       -1.049991 -0.989983 -1.059754 -1.121333    ...     0.726206   

condition                                                              \
time           600       650       700       750       800       850    
G1        -0.030548 -0.013574  0.294615  0.192319  0.891598  1.367119   
G2        -1.192432 -2.087052 -0.885982  0.114952  0.890671  0.107399   
G3        -1.801480 -0.720565 -0.627976  0.091069  0.451285  0.161373   
G4         3.186237  2.331540 -0.271496  0.021489 -0.317333 -0.300106   
G5        -0.351648 -0.799730 -0.721109 -1.010996 -1.034908 -0.588070   
G6         0.591435 -0.242654 -0.798989 -0.859009 -0.973947 -0.929852   
G7         0.786504  0.391701 -0.191543 -0.539719 -0.449491 -1.021770   
G8        -0.533814 -0.734053  0.420854  0.707833  0.807185  0.679407   
G9         0.776734  0.730794  1.319809  0.135237 -0.307959 -0.823089   
G10        0.622430  1.082830  1.147397  1.157381  0.821923  0.834685   

condition                                
time           900       950       1000  
G1         1.405764  1.575214  1.677220  
G2         0.103234 -0.062799  2.047518  
G3         0.883697  1.027753  0.114507  
G4        -1.006657 -0.399047 -0.630055  
G5        -1.355346 -1.261162 -1.091058  
G6        -1.013540 -0.971334 -1.010994  
G7        -0.032347 -0.912598 -1.018173  
G8         0.935787  1.125172  1.745371  
G9        -0.675054 -0.974258 -0.548550  
G10        0.726725  0.706422  1.465279  

[10 rows x 21 columns]

In [100]:
csidata

condition         A                                                    \
time           0         50        100       150       200       250    
G1         1.146141 -0.831964 -1.439418 -1.737505 -1.749940 -1.756526   
G2        -0.291546  2.122344  1.711602 -0.043294  0.554139  1.055807   
G3         1.762955  1.328581  0.812283  0.977008  0.497170  0.238698   
G4        -0.761209 -0.790224 -0.757698 -0.750799 -0.735785 -0.350352   
G5        -0.626803  0.283999  0.150912  0.460401  0.334145  0.272127   
G6        -0.859092 -0.973677 -0.500515 -0.023210  0.519307  0.808823   
G7        -0.654526 -0.656295 -0.687107 -0.678800 -0.539539 -0.544813   
G8         0.813293  1.454814  0.487546  0.283878 -0.178654 -0.622127   
G9        -0.921218 -0.971412 -1.050051 -1.362399 -0.997308 -0.742705   
G10        0.454779 -1.367237 -1.763047 -1.945656 -1.997908 -2.110899   

condition                                            ...            E  \
time           300       350       400       450     ...         550    
G1        -1.765812 -1.769060 -1.760694 -1.759075    ...     1.514828   
G2         0.679701  0.786423  0.448707 -0.105750    ...     0.261458   
G3         0.045474  0.304781 -0.127848  0.642854    ...     0.198277   
G4        -0.641694 -0.265993 -0.757629 -0.322865    ...    -0.442139   
G5         0.280810  0.390443  0.485875  0.445135    ...    -0.494777   
G6         1.028765  1.231111  1.315269  1.422283    ...    -0.302052   
G7        -0.173020 -0.097119 -0.090761  0.042880    ...    -0.199461   
G8        -0.883580 -0.554871 -0.800245 -0.777222    ...    -0.475700   
G9        -0.155355  0.624382  1.433256  1.058768    ...     0.116064   
G10       -1.964892 -1.882316 -1.978326 -2.063066    ...     0.242996   

condition                                                              \
time           600       650       700       750       800       850    
G1         0.534294 -0.024755 -0.126884  0.600194  0.914042  0.786232   
G2         0.493261  0.377926  0.753657  0.385389  0.574857  0.281021   
G3         1.033614  0.204734  1.029577  0.853479  0.498641  0.719518   
G4        -0.642049 -0.146133 -0.377510 -0.774507 -0.786612 -0.219383   
G5        -0.257971 -0.512780 -0.653853 -0.469474 -0.591243 -0.697342   
G6        -0.742568 -0.592456 -0.749805 -0.901809 -0.858331 -0.881814   
G7         0.085773 -0.518966 -0.526747 -0.592528 -0.560815 -0.575871   
G8         0.281614  0.589848  1.161354  0.836138  1.216442  0.984242   
G9         1.098921  0.282571 -0.432951 -0.723064 -0.921609 -0.913787   
G10        0.364398  0.499968  0.172150  0.541871  0.641857  0.955349   

condition                                
time           900       950       1000  
G1         1.186814  1.242868  0.904182  
G2         1.169579  0.856793  0.597477  
G3         0.332503  0.348764  0.859902  
G4        -0.745372 -0.752894 -0.737860  
G5        -0.451460 -0.530345 -0.429491  
G6        -0.895345 -0.897960 -0.932892  
G7        -0.693039 -0.693202 -0.681000  
G8         1.433824  1.324359  2.088668  
G9        -0.725886 -0.834355 -0.843264  
G10        1.089855  0.708586  1.639483  

[10 rows x 105 columns]

In [101]:
        inp = csidata.iloc[:,numtemp[colNames=='A']]

In [104]:
inp.loc[:,:] = sp.stats.mstats.zscore(inp,axis=1,ddof=1)

//anaconda/lib/python3.4/site-packages/pandas/core/indexing.py:420: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [105]:
        inp = csidata.iloc[:,numtemp[colNames=='A']]

In [106]:
            data_standardised = sp.stats.mstats.zscore(inp,axis=1,ddof=1)
            inp.iloc[:,:] = data_standardised

//anaconda/lib/python3.4/site-packages/pandas/core/indexing.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
//anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [108]:
            data_standardised = sp.stats.mstats.zscore(inp,axis=1,ddof=1)
            inp.loc[inp.index,inp.columns.values] = data_standardised

//anaconda/lib/python3.4/site-packages/pandas/core/indexing.py:420: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [109]:
            data_standardised = sp.stats.mstats.zscore(inp,axis=1,ddof=1)
            for i in np.arange(inp.shape[0]):
                for j in np.arange(inp.shape[1]):
                    inp.iloc[i,j] = data_standardised[i,j]

//anaconda/lib/python3.4/site-packages/pandas/core/indexing.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
//anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [110]:
            data_standardised = sp.stats.mstats.zscore(inp,axis=1,ddof=1)
            inp[:][:] = data_standardised

In [111]:
inp

condition         A                                                    \
time           0         50        100       150       200       250    
G1         1.457348 -0.181595 -0.684896 -0.931874 -0.942176 -0.947633   
G2        -0.947806  1.801261  1.333486 -0.665083  0.015306  0.586631   
G3         1.900531  1.297516  0.580768  0.809447  0.143316 -0.215506   
G4        -0.587374 -0.642793 -0.580669 -0.567492 -0.538813  0.197373   
G5        -1.141544  0.828327  0.540489  1.209848  0.936783  0.802652   
G6        -0.969230 -1.091449 -0.586762 -0.077659  0.501003  0.809808   
G7        -0.899677 -0.906224 -1.020254 -0.989509 -0.474129 -0.493646   
G8         0.804977  1.587897  0.407431  0.158872 -0.405608 -0.946827   
G9        -0.889359 -0.945905 -1.034494 -1.386365 -0.975077 -0.688258   
G10        0.708361 -0.615681 -0.903312 -1.036012 -1.073984 -1.156093   

condition                                            ...               \
time           300       350       400       450     ...         550    
G1        -0.955327 -0.958018 -0.951086 -0.949745    ...    -0.375141   
G2         0.158302  0.279842 -0.104767 -0.736211    ...    -0.995543   
G3        -0.483748 -0.123766 -0.724361  0.345561    ...    -2.468511   
G4        -0.359098  0.358499 -0.580537  0.249874    ...     0.416273   
G5         0.821430  1.058544  1.264942  1.176831    ...    -0.465723   
G6         1.044403  1.260230  1.349995  1.464139    ...     0.905768   
G7         0.882302  1.163199  1.186729  1.681313    ...     2.208214   
G8        -1.265907 -0.864747 -1.164204 -1.136106    ...    -0.983780   
G9        -0.026587  0.851815  1.763040  1.341167    ...     0.941079   
G10       -1.049991 -0.989983 -1.059754 -1.121333    ...     0.726206   

condition                                                              \
time           600       650       700       750       800       850    
G1        -0.030548 -0.013574  0.294615  0.192319  0.891598  1.367119   
G2        -1.192432 -2.087052 -0.885982  0.114952  0.890671  0.107399   
G3        -1.801480 -0.720565 -0.627976  0.091069  0.451285  0.161373   
G4         3.186237  2.331540 -0.271496  0.021489 -0.317333 -0.300106   
G5        -0.351648 -0.799730 -0.721109 -1.010996 -1.034908 -0.588070   
G6         0.591435 -0.242654 -0.798989 -0.859009 -0.973947 -0.929852   
G7         0.786504  0.391701 -0.191543 -0.539719 -0.449491 -1.021770   
G8        -0.533814 -0.734053  0.420854  0.707833  0.807185  0.679407   
G9         0.776734  0.730794  1.319809  0.135237 -0.307959 -0.823089   
G10        0.622430  1.082830  1.147397  1.157381  0.821923  0.834685   

condition                                
time           900       950       1000  
G1         1.405764  1.575214  1.677220  
G2         0.103234 -0.062799  2.047518  
G3         0.883697  1.027753  0.114507  
G4        -1.006657 -0.399047 -0.630055  
G5        -1.355346 -1.261162 -1.091058  
G6        -1.013540 -0.971334 -1.010994  
G7        -0.032347 -0.912598 -1.018173  
G8         0.935787  1.125172  1.745371  
G9        -0.675054 -0.974258 -0.548550  
G10        0.726725  0.706422  1.465279  

[10 rows x 21 columns]

In [116]:
np.std(inp.iloc[0,:],ddof=1)

1.0000000000000002

In [115]:
            inp[:][:] = sp.stats.mstats.zscore(inp,axis=1,ddof=1)